# Pandas

TensorFlow2 공식 홈페이지에 나와 있는 코드들을 수행해 보면서 빠르게 TensorFlow2를 익혀 봅니다.

https://www.tensorflow.org/tutorials/load_data/pandas_dataframe

##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
# 경고 메시지 출력 끄기
import warnings 
warnings.filterwarnings(action='ignore')

# 노트북 셀 표시를 브라우저 전체 폭 사용하기
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import clear_output

%matplotlib inline
import matplotlib.pyplot as plt

import os, sys, shutil, functools
import collections, pathlib, re, string

rseed = 22
import random
random.seed(rseed)

import numpy as np
np.random.seed(rseed)
np.set_printoptions(precision=5)
np.set_printoptions(formatter={'float_kind': "{:.5f}".format})

import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.5f}'.format

import tensorflow as tf
tf.random.set_seed(rseed)
tf.keras.backend.set_floatx('float32') # keras default float type 설정

import tensorflow_hub as tfhub
import tensorflow_datasets as tfds
import tensorflow_text as tftext

import kerastuner as kt

import seaborn as sns

import PIL
import PIL.Image

print(f"python ver={sys.version}")
print(f"pandas ver={pd.__version__}")
print(f"numpy ver={np.__version__}")
print(f"tensorflow ver={tf.__version__}")
print(f"tensorflow execuring eagerly={tf.executing_eagerly()}")
print(f"tensorflow GPU={'True' if tf.config.experimental.list_physical_devices('GPU') else 'False'}")
print(f"keras tuner ver={kt.__version__}")
print(f"senborn ver={sns.__version__}")
print(f"PIL ver={PIL.__version__}")

python ver=3.7.6 (default, Nov 21 2020, 22:51:13) 
[Clang 12.0.0 (clang-1200.0.32.27)]
pandas ver=1.0.5
numpy ver=1.19.5
tensorflow ver=2.4.1
tensorflow execuring eagerly=True
tensorflow GPU=False
keras tuner ver=1.0.2
senborn ver=0.10.0
PIL ver=7.1.2


## Load a pandas.DataFrame

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/pandas_dataframe"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
*

____
This tutorial provides an example of how to load pandas dataframes into a `tf.data.Dataset`.

This tutorials uses a small [dataset](https://archive.ics.uci.edu/ml/datasets/heart+Disease) provided by the Cleveland Clinic Foundation for Heart Disease. There are several hundred rows in the CSV. Each row describes a patient, and each column describes an attribute. We will use this information to predict whether a patient has heart disease, which in this dataset is a binary classification task.

## Read data using pandas

Download the csv file containing the heart dataset.

In [2]:
csv_file = tf.keras.utils.get_file('/tmp/heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


Read the csv file using pandas.

In [9]:
df = pd.read_csv(csv_file)

In [10]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.30000,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.50000,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.60000,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.50000,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.40000,1,0,normal,0


In [11]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

Convert `thal` column which is an `object` in the dataframe to a discrete numerical value.

In [12]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [13]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.30000,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.50000,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.60000,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.50000,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.40000,1,0,3,0


## Load data using `tf.data.Dataset`

Use `tf.data.Dataset.from_tensor_slices` to read the values from a pandas dataframe. 

One of the advantages of using `tf.data.Dataset` is it allows you to write simple, highly efficient data pipelines. Read the [loading data guide](https://www.tensorflow.org/guide/data) to find out more.

In [14]:
target = df.pop('target')

In [15]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [16]:
for feat, targ in dataset.take(5):
    print ('Features: {}, Target: {}'.format(feat, targ))

Features: [63.00000 1.00000 1.00000 145.00000 233.00000 1.00000 2.00000 150.00000
 0.00000 2.30000 3.00000 0.00000 2.00000], Target: 0
Features: [67.00000 1.00000 4.00000 160.00000 286.00000 0.00000 2.00000 108.00000
 1.00000 1.50000 2.00000 3.00000 3.00000], Target: 1
Features: [67.00000 1.00000 4.00000 120.00000 229.00000 0.00000 2.00000 129.00000
 1.00000 2.60000 2.00000 2.00000 4.00000], Target: 0
Features: [37.00000 1.00000 3.00000 130.00000 250.00000 0.00000 0.00000 187.00000
 0.00000 3.50000 3.00000 0.00000 3.00000], Target: 0
Features: [41.00000 0.00000 2.00000 130.00000 204.00000 0.00000 2.00000 172.00000
 0.00000 1.40000 1.00000 0.00000 3.00000], Target: 0


Since a `pd.Series` implements the `__array__` protocol it can be used transparently nearly anywhere you would use a `np.array` or a `tf.Tensor`.

In [17]:
tf.constant(df['thal'])

<tf.Tensor: shape=(303,), dtype=int8, numpy=
array([2, 3, 4, 3, 3, 3, 3, 3, 4, 4, 2, 3, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3,
       3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 4, 2, 4, 3, 4, 3, 4, 4,
       2, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4,
       4, 2, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 4, 4, 4,
       3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4,
       3, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 3, 2, 4, 4, 2, 3, 3, 4, 4, 3, 4,
       3, 3, 4, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2,
       4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 4, 3, 4, 2, 4, 3,
       3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 2, 2, 4, 3, 4, 3, 2, 4, 3, 3, 2,
       4, 4, 4, 4, 3, 0, 3, 3, 3, 3, 1, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4,
       3, 3, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3, 3,
      

Shuffle and batch the dataset.

In [18]:
train_dataset = dataset.shuffle(len(df)).batch(1)

## Create and train a model

In [19]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

In [20]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Epoch 1/15
303/303 [==============================] - 1s 568us/step - loss: 2.3896 - accuracy: 0.5740
Epoch 2/15
303/303 [==============================] - 0s 575us/step - loss: 0.8517 - accuracy: 0.7286
Epoch 3/15
303/303 [==============================] - 0s 573us/step - loss: 0.6707 - accuracy: 0.7028
Epoch 4/15
303/303 [==============================] - 0s 608us/step - loss: 0.5846 - accuracy: 0.7629
Epoch 5/15
303/303 [==============================] - 0s 568us/step - loss: 0.5155 - accuracy: 0.7901
Epoch 6/15
303/303 [==============================] - 0s 568us/step - loss: 0.5284 - accuracy: 0.7841
Epoch 7/15
303/303 [==============================] - 0s 570us/step - loss: 0.5085 - accuracy: 0.7598
Epoch 8/15
303/303 [==============================] - 0s 572us/step - loss: 0.6247 - accuracy: 0.6896
Epoch 9/15
303/303 [==============================] - 0s 576us/step - loss: 0.5508 - accuracy: 0.7335
Epoch 10/15
303/303 [==============================] - 0s 572us/step - loss: 0.508

## Alternative to feature columns

Passing a dictionary as an input to a model is as easy as creating a matching dictionary of `tf.keras.layers.Input` layers, applying any pre-processing and stacking them up using the [functional api](../../guide/keras/functional.ipynb). You can use this as an alternative to [feature columns](../keras/feature_columns.ipynb).

In [21]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

The easiest way to preserve the column structure of a `pd.DataFrame` when used with `tf.data` is to convert the `pd.DataFrame` to a `dict`, and slice that dictionary.

In [22]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [23]:
for dict_slice in dict_slices.take(1):
    print (dict_slice)

({'age': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57],
      dtype=int32)>, 'sex': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)>, 'cp': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3], dtype=int32)>, 'trestbps': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150], dtype=int32)>, 'chol': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168], dtype=int32)>, 'fbs': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int32)>, 'restecg': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0], dtype=int32)>, 'thalach': <tf.Tensor: shape=(16

In [24]:
model_func.fit(dict_slices, epochs=15)

Epoch 1/15
19/19 [==============================] - 0s 1ms/step - loss: 51.7932 - accuracy: 0.2507
Epoch 2/15
19/19 [==============================] - 0s 1ms/step - loss: 38.8308 - accuracy: 0.2507
Epoch 3/15
19/19 [==============================] - 0s 1ms/step - loss: 25.2137 - accuracy: 0.2560
Epoch 4/15
19/19 [==============================] - 0s 1ms/step - loss: 10.8219 - accuracy: 0.3579
Epoch 5/15
19/19 [==============================] - 0s 1ms/step - loss: 3.5714 - accuracy: 0.6335
Epoch 6/15
19/19 [==============================] - 0s 1ms/step - loss: 3.0396 - accuracy: 0.7098
Epoch 7/15
19/19 [==============================] - 0s 1ms/step - loss: 2.9735 - accuracy: 0.7215
Epoch 8/15
19/19 [==============================] - 0s 1ms/step - loss: 2.8860 - accuracy: 0.7103
Epoch 9/15
19/19 [==============================] - 0s 1ms/step - loss: 2.7990 - accuracy: 0.7022
Epoch 10/15
19/19 [==============================] - 0s 1ms/step - loss: 2.7070 - accuracy: 0.7022
Epoch 11/15
19/